In [2]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10

physical_devices = tf.config.list_physical_devices('GPU')
print('Physical Devices', physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Physical Devices [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Load Data

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 16s 0us/step


In [4]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

### Returning a Functional model from Functions

The regularization for each layer can be added to each layer through the kernel_regularizer paramater in the layer constructor. For each layer, we can assign regularization method by passing it to the function like below. 
<br>
<br>
Dropout by itself is a layer present in layers.Dropout and it accepts the dropout probability as a parameter. The output of a layer is passed to a Dropout layer like below. 

In [5]:
def my_model():
    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(64, 5, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(128, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)

    x = layers.Flatten()(x)
    
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [6]:
model = my_model()

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        51264 

In [8]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=3e-4),
    metrics=['accuracy']
)

In [9]:
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)

Epoch 1/10
782/782 - 9s - loss: 3.0788 - accuracy: 0.3309 - 9s/epoch - 12ms/step
Epoch 2/10
782/782 - 5s - loss: 1.9180 - accuracy: 0.4477 - 5s/epoch - 7ms/step
Epoch 3/10
782/782 - 6s - loss: 1.6164 - accuracy: 0.5101 - 6s/epoch - 7ms/step
Epoch 4/10
782/782 - 5s - loss: 1.4900 - accuracy: 0.5448 - 5s/epoch - 7ms/step
Epoch 5/10
782/782 - 5s - loss: 1.4315 - accuracy: 0.5612 - 5s/epoch - 7ms/step
Epoch 6/10
782/782 - 6s - loss: 1.3864 - accuracy: 0.5787 - 6s/epoch - 7ms/step
Epoch 7/10
782/782 - 5s - loss: 1.3489 - accuracy: 0.5930 - 5s/epoch - 7ms/step
Epoch 8/10
782/782 - 5s - loss: 1.3231 - accuracy: 0.6042 - 5s/epoch - 7ms/step
Epoch 9/10
782/782 - 6s - loss: 1.3093 - accuracy: 0.6106 - 6s/epoch - 7ms/step
Epoch 10/10
782/782 - 5s - loss: 1.2944 - accuracy: 0.6153 - 5s/epoch - 7ms/step


In [10]:
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

157/157 - 1s - loss: 1.4646 - accuracy: 0.5574 - 946ms/epoch - 6ms/step


[1.4645832777023315, 0.5573999881744385]